# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [1]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet

import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/mikedemayo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mikedemayo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/mikedemayo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/mikedemayo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("../pset3/combined.json", lines=True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [5]:
pharma_series = doj.loc[doj['id'] == '17-1204', 'contents']
pharma = pharma_series.iloc[0] if not pharma_series.empty else ''

# Tokenize and remove punctuation/numbers
tokens = word_tokenize(pharma.lower())
alpha_tokens = [word for word in tokens if word.isalpha()]
# removing string punctuation

# Show first 10 clean tokens
print(alpha_tokens[:10])

['the', 'founder', 'and', 'majority', 'owner', 'of', 'insys', 'therapeutics', 'was', 'arrested']


### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [6]:
# Tag each word with its part of speech
tagged_tokens = pos_tag(alpha_tokens)

# Show the first 10 tagged tokens to make sure it works
tagged_tokens[:10]



[('the', 'DT'),
 ('founder', 'NN'),
 ('and', 'CC'),
 ('majority', 'NN'),
 ('owner', 'NN'),
 ('of', 'IN'),
 ('insys', 'JJ'),
 ('therapeutics', 'NNS'),
 ('was', 'VBD'),
 ('arrested', 'VBN')]

In [27]:
from collections import Counter

# Tags that represent adjectives in NLTK
adjective_tags = {'JJ'}

# Extract all adjectives from the tagged tokens
adjectives = [word for word, tag in tagged_tokens if tag in adjective_tags]

# Count their frequencies
adj_counts = Counter(adjectives)

# Convert to DataFrame and sort
adj_df = pd.DataFrame(adj_counts.items(), columns=['adjective', 'count'])
adj_df = adj_df.sort_values(by='count', ascending=False).head(5)

# Show the top 5 adjectives
adj_df

## former 8, opiod 5, nationwide 4, addictive 3, other 3



,adjective,count
13,former,8
41,special,7
37,opioid,5
55,criminal,5
73,postal,4


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [28]:
# Apply spaCy's NER pipeline to the original (unprocessed) press release
doc = nlp(pharma)

# Extract all named entities as (text, label) pairs
all_ents = [(ent.text, ent.label_) for ent in doc.ents]

# Preview sample
all_ents[:10]



[('Insys Therapeutics Inc.', 'ORG'),
 ('today', 'DATE'),
 ('Fentanyl', 'PERSON'),
 ('More than 20,000', 'CARDINAL'),
 ('Americans', 'NORP'),
 ('last year', 'DATE'),
 ('millions', 'CARDINAL'),
 ('Jeff Sessions', 'PERSON'),
 ('This Justice Department', 'ORG'),
 ('Trump', 'PERSON')]

In [29]:
law_ents = sorted(set(ent.text for ent in doc.ents if ent.label_ == 'LAW'))

# Print them 
print("Unique LAW entities:")
for ent in law_ents:
    print("-", ent)

Unique LAW entities:
- RICO
- the Controlled Substances Act


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

In [30]:
## Rico is a U.S. law made to combat organized crime by allowing prosecution of individuals or groups involved in a 
##pattern of racketeering activity. This isapplicable to pharmaceutical cases where executives allegedly ran the company 
#like a criminal enterprise to profit from illegal schemes such as prescribing kickbacks.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [31]:
## your code here
# Find "year" or "years"
year_dates = [
    ent.text for ent in doc.ents
    if ent.label_ == "DATE" and re.search(r"\byears?\b", ent.text.lower())
]

# Print them
print("DATE entities that mention years:")
for ent in year_dates:
    print("-", ent)

# print ones that say date as well **


DATE entities that mention years:
- last year
- 20 years
- three years
- five years
- three years


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [32]:
## your code here

sentences = sent_tokenize(pharma)

# Step 2: Extract sentences mentioning "years"
sentences_with_years = [s for s in sentences if re.search(r'\b\d+\s+years?\b', s.lower())]

# Step 3: Print the matching sentences
print("Sentences mentioning prison/probation durations:\n")
for s in sentences_with_years:
    print("-", s.strip(), "\n")



# If convicted, the CEO may face up to 20 years in prison and up to 3 years of supervised release. 
# res = [p for p in pharma.split(".") if 'year' is in p]



Sentences mentioning prison/probation durations:

- This investigation highlights our commitment to defending our mail system from illegal misuse and ensuring public trust in the mail.”“The U.S. Department of Veterans Affairs, Office of Inspector General will continue to aggressively investigate those that attempt to fraudulently impact programs designed to benefit our veterans and their families,” said Donna L. Neves, Special Agent in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss. 



## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [33]:
## your code here for subsetting
topics_clean = ["Civil Rights", "Hate Crimes", "Project Safe Childhood"]
doj_subset = doj[doj['topics_clean'].apply(
    lambda x: x.strip() in topics_clean
)]

print("Rows in doj_subset:", len(doj_subset))


Rows in doj_subset: 717


B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [34]:
## your code here to define function

sia = SentimentIntensityAnalyzer()

# Step 4: Define the function
def clean_and_score_sentiment(text):
    # Remove named entities (simple capitalized phrases, 1-5 words)
    cleaned = re.sub(r'\b(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+){0,4})\b', '', text)
    # ADD CLEAN PUNCTUATION
    
    # Score the sentiment
    return sia.polarity_scores(cleaned)


In [35]:
## your code here executing the function
doj_subset['sentiment'] = doj_subset['contents'].apply(clean_and_score_sentiment)
print(clean_and_score_sentiment(pharma))
#0.173

{'neg': 0.15, 'neu': 0.75, 'pos': 0.1, 'compound': -0.9977}


C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [36]:
## your code here
sentiment_df = doj_subset['sentiment'].apply(pd.Series)
doj_subset_wscore = pd.concat([doj_subset, sentiment_df], axis=1)

# Sort from highest to lowest negative score
doj_subset_wscore = doj_subset_wscore.sort_values(by='neg', ascending=False)


doj_subset_wscore.head(2)

,id,title,contents,date,topics_clean,components_clean,sentiment,neg,neu,pos,compound
329,14-248,Albuquerque Man Charged with Federal Hate Crime Related to Anti-Semitic Threats Against Businesswoman,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",2014-03-10T00:00:00-04:00,Hate Crimes,Civil Rights Division; Civil Rights - Criminal Section,"{'neg': 0.319, 'neu': 0.652, 'pos': 0.029, 'compound': -0.9957}",0.319,0.652,0.029,-0.9957
11593,16-718,Three Mississippi Correctional Officers Indicted for Inmate Assault and Cover-Up,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",2016-06-21T00:00:00-04:00,Civil Rights,"Civil Rights Division; Civil Rights - Criminal Section; USAO - Mississippi, Northern","{'neg': 0.288, 'neu': 0.68, 'pos': 0.032, 'compound': -0.9964}",0.288,0.680,0.032,-0.9964


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [37]:
## agg and find the mean compound score by topic
# Check, errors in getting double compound
doj_subset_wscore = doj_subset_wscore.drop(columns=["compound"], errors="ignore")

doj_subset_wscore["compound"] = doj_subset_wscore["sentiment"].apply(
    lambda x: x.get("compound") if isinstance(x, dict) else None)

# Step 2: Group by topic and calculate mean compound sentiment
mean_sentiment_by_topic = doj_subset_wscore.groupby("topics_clean")["compound"].mean()

# Step 3: Print result
print(mean_sentiment_by_topic)

topics_clean
Civil Rights             -0.106531
Hate Crimes              -0.934456
Project Safe Childhood   -0.722572
Name: compound, dtype: float64


 ## ADD SENTENCE HERE FOR DESCRIPTION

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [38]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [39]:
# Setting up snowball stemmer and combining stopwords into master list
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english")).union(custom_doj_stopwords)

# Writing function for preprocessing each contents col string
def preprocess_contents(some_string):
    text = some_string.lower()
    # Tokenizing the contents section
    tokens = word_tokenize(text)
    # Filtering words
    filtered = [word for word in tokens if word.isalpha() and word not in stop_words and len(word) >= 4]

    stemmed = [stemmer.stem(word) for word in filtered]
    # Returning joined string
    return " ".join(stemmed)

    

In [40]:
# Applying the function onto the dataframe
doj_subset_wscore["processed_text"] = doj_subset_wscore["contents"].apply(preprocess_contents)


In [41]:
# showing examples based on the IDs given
selected_ids = ["16-718", "16-217"]

#Filtering
doj_subset_wscore.loc[doj_subset_wscore["id"].isin(selected_ids), ["id", "contents", "processed_text"]]

,id,contents,processed_text
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi dana mulhaus crimin section marsher indict
6727,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved sh

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [42]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [43]:
# Using create_dtm function, using preprocessed releases w/ cols id, compound, sentiment, topics_clean
metadata = doj_subset_wscore[["id", "compound", "topics_clean"]]

dtm = create_dtm(doj_subset_wscore["processed_text"], metadata) 
# to avoid same col names
dtm = dtm.loc[:, ~dtm.columns.duplicated()]

In [44]:
# writing function get_topwords
def get_topwords(dtm_subset):
    # Get only word columns
    word_cols = dtm_subset.columns.difference(["id", "compound", "topics_clean"])
    
    # Sum frequencies
    word_sums = dtm_subset[word_cols].sum().sort_values(ascending=False)
    
    return word_sums.head(10)

# Getting top and bottom 5% of compound scores
top_5 = doj_subset_wscore["compound"].quantile(0.95)
bottom_5 = doj_subset_wscore["compound"].quantile(0.05)

# Setting Subsets
dtm_top5 = dtm[dtm["compound"] >= top_5]
dtm_bottom5 = dtm[dtm["compound"] <= bottom_5]

In [45]:
print("Top 10 words in top 5% positive sentiment:")
print(get_topwords(dtm_top5))


Top 10 words in top 5% positive sentiment:
index         240977
agreement        176
state            123
enforc           116
ensur            105
disabl           102
polic             88
settlement        88
student           86
general           83
dtype: int64


In [46]:
print("\nTop 10 words in bottom 5% negative sentiment:")
print(get_topwords(dtm_bottom5))


Top 10 words in bottom 5% negative sentiment:
index      314621
assault       202
victim        185
crime         168
offic         164
hate          135
defend        130
sentenc       111
charg         101
feder         101
dtype: int64


## D. Print the top 10 words for press releases in each of the three topics_clean
# add here

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [47]:
# Make list of words
token_lists = [doc.split() for doc in doj_subset_wscore["processed_text"]]

In [48]:
# Creating dictionary
dictionary = corpora.Dictionary(token_lists)

# Turning into bag of words format
corpus = [dictionary.doc2bow(tokens) for tokens in token_lists]

In [49]:
# Fit LDA model from imports at start of code
lda_model = gensim.models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=3,
    random_state=42,       
    passes=6,             
    alpha='auto',          
    per_word_topics=True   
)

In [50]:
# Print out top 15 words in each topic
for i, topic in lda_model.show_topics(num_topics = 3, num_words = 15, formatted = False):
    print(f"\nTopic {i + 1}:")
    print(", ".join([word for word, _ in topic]))


Topic 1:
child, exploit, sexual, safe, offic, project, childhood, children, crimin, pornographi, state, prosecut, sentenc, victim, investig

Topic 2:
victim, charg, prosecut, feder, sentenc, defend, crime, guilti, said, indict, hate, prison, year, investig, today

Topic 3:
hous, disabl, discrimin, said, enforc, court, alleg, fair, today, violat, general, individu, access, settlement, feder


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [51]:
## your code here to get doc-level topic probabilities 
doc_topics = [lda_model.get_document_topics(bow, minimum_probability = 0) for bow in corpus]
assert len(doc_topics) == len(doj_subset_wscore), "Mismatch in number of topic distributions"

In [52]:
## your code here to add those topic probabilities to the dataframe
# turning into wide format and renaming cols
topic_probs = pd.DataFrame([[prob for _, prob in doc] for doc in doc_topics]) # check probability calc (np.argmax)
topic_probs.columns = ["lda_topic_1", "lda_topic_2", "lda_topic_3"]

# Concat together
doj_subset_wscore = doj_subset_wscore.join(topic_probs)

# Creating top_topic col
doj_subset_wscore["top_topic"] = topic_probs.idxmax(axis=1)

# .032, .64, .31
# .13, .09, .77
#.99, 0, .00

In [53]:
## your code here to summarize the topic proportions for each of the topics_clean
topic_breakdown = pd.crosstab(
    doj_subset_wscore["topics_clean"],
    doj_subset_wscore["top_topic"],
    normalize='index'  
)

# Converting vals to percentages
topic_breakdown = (topic_breakdown * 100).round(1)
topic_breakdown

top_topic,lda_topic_1,lda_topic_2,lda_topic_3
topics_clean,,,
Civil Rights,66.7,33.3,0.0
Hate Crimes,50.0,31.8,18.2
Project Safe Childhood,54.5,36.4,9.1


## ***NEED SENTENCE w/ EXAMPLES HERE FOR PART D IF RIGHT

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [56]:
# Part A:
# Creating function
def create_bigram_onedoc(text):
    words = text.split()
    bigrams = [f"{w1}_{w2}" for w1, w2 in zip(words, words[1:])]
    return " ".join(bigrams)

# Applying to data frame
doj_subset_wscore["processed_text_bigram"] = doj_subset_wscore["processed_text"].apply(create_bigram_onedoc)

# Part B, printing out results:
doj_subset_wscore.loc[doj_subset_wscore["id"] == "16-217", ["id", "processed_text", "processed_text_bigram"]]

,id,processed_text,processed_text_bigram
6727,16-217,reach comprehens settlement agreement citi miami miami polic resolv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim shoot effect quick investig shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut citizen said ferrer oversight communic agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_general general_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_shoot shoot_effect effect_quick quick_investig investig_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_general general_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_today today_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_citizen citizen_said said_ferrer ferrer_oversight oversight_communic communic_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_made made_applaud applaud_citi citi_commiss commiss_settlement settlement_agreement agreement_build build_upon upon_import import_reform reform_implement implement_citi citi_sinc sinc_issu issu_find find

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [57]:
# creating another document-term matrix
metadata_bigram = doj_subset_wscore[["id", "topics_clean", "compound"]]

dtm_bigram = create_dtm(doj_subset_wscore["processed_text_bigram"], metadata_bigram)

In [58]:
# Printing dimensions of matrices to compare
print("Unigram DTM dimensions:", dtm.shape)         
print("Bigram DTM dimensions:", dtm_bigram.shape)
# 6914, 66649

dtm_bigram

Unigram DTM dimensions: (717, 6759)
Bigram DTM dimensions: (717, 71336)


,index,id,topics_clean,compound,aaron_ford,aaron_latham,aaron_mcgrath,aaron_parrish,aaron_polster,aaron_rice,...,zone_request,zone_restrict,zone_student,zone_time,zone_today,zone_varianc,zunggeemog_noel,zunggeemog_prompt,zunggeemog_write,zwengel_princeton
0,329,14-248,Hate Crimes,-0.9957,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11593,16-718,Civil Rights,-0.9964,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,572,13-312,Hate Crimes,-0.9977,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,501,11-626,Hate Crimes,-0.9985,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11903,18-70,Civil Rights,-0.9908,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,7594,16-539,Civil Rights,0.9854,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,6787,17-132,Civil Rights,0.9758,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,6981,17-003,Civil Rights,0.9766,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,1857,17-271,Civil Rights,0.9899,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Why is the bigram matrix has more dimensions?
- dtm_migram has more dimensions because the bigram contains more than one word unlike the individual words before
- In that way the vocabulary can increase. Combining two words together can make a place, or known individual
- Thus, leading to more columns than before with just a singular word

In [64]:
# Finding and printing the 10 most prevelant bigrams for each of the three topics_clean
# Subsetting for each topic
topic1_bigram = dtm_bigram[dtm_bigram["topics_clean"] == "Hate Crimes"]
topic2_bigram = dtm_bigram[dtm_bigram["topics_clean"] == "Civil Rights"]
topic3_bigram = dtm_bigram[dtm_bigram["topics_clean"] == "Project Safe Childhood"]

# Calling and printing on each one
print("Top 10 bigrams for Hate Crimes:")
print(get_topwords(topic1_bigram))

print("\nTop 10 bigrams for Civil Right:")
print(get_topwords(topic2_bigram))

print("\nTop 10 bigrams for Project Safe Childhood:")
print(get_topwords(topic3_bigram))

Top 10 bigrams for Hate Crimes:
index            1972570
hate_crime           368
plead_guilti         275
year_prison          160
special_agent        117
thoma_perez          111
grand_juri           101
perez_general         95
said_thoma            91
unit_state            87
dtype: int64

Top 10 bigrams for Civil Right:
index             1941047
fair_hous             227
princip_deputi        221
deputi_general        221
vanita_gupta          202
gupta_head            200
general_vanita        199
said_princip          186
unit_state            152
nation_origin         141
dtype: int64

Top 10 bigrams for Project Safe Childhood:
index                1155767
project_safe             472
safe_childhood           472
child_pornographi        445
child_exploit            279
sexual_exploit           223
exploit_children         199
plead_guilti             197
child_sexual             175
exploit_obscen           175
dtype: int64


# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 